In [44]:
#Imports
import pandas as pd
import os
from sklearn.linear_model import LinearRegression
from geopy.geocoders import Nominatim


In [45]:
# Read the CSV file into a DataFrame
allproperties = pd.read_csv('Data/original_data/realtor-data.zip.csv')
#allproperties.dtypes
#new data directory
output_dir = 'Data/new_data'
os.makedirs(output_dir, exist_ok=True)
# Directory containing the newly created CSV files
input_dir = 'Data/new_data'

In [46]:
# Add the 'sold_previously' column based on 'prev_sold_date'
allproperties['sold_previously'] = allproperties['prev_sold_date'].notna().astype(int)
# Drop the 'prev_sold_date' column
allproperties = allproperties.drop(columns=['prev_sold_date', 'status'])
# Fill missing values in 'bed' and 'bath' columns with 1
allproperties['bed'].fillna(1, inplace=True)
allproperties['bath'].fillna(1, inplace=True)
# Drop rows with NaNs in the 'zip_code' column
allproperties.dropna(subset=['zip_code', 'price', 'city'], inplace=True)
# Drop rows with 'price' less than 2000
allproperties = allproperties[allproperties['price'] >= 1000]
# Drop rows with 'house_size' less than or equal to 0
allproperties = allproperties[allproperties['house_size'] > 0]
# Drop rows with 'acre_lot' less than or equal to 0
allproperties = allproperties[allproperties['acre_lot'] > 0]
# Multiply negative values in 'acre_lot' column by -1 to make them positive
allproperties['acre_lot'] = allproperties['acre_lot'].apply(lambda x: x * -1 if x < 0 else x)

In [47]:
#remove outliers from specific columns

# Calculate the IQR for each column
Q1 = allproperties[['price', 'bed', 'house_size', 'acre_lot', 'bath']].quantile(0.25)
Q3 = allproperties[['price', 'bed', 'house_size', 'acre_lot', 'bath']].quantile(0.75)
IQR = Q3 - Q1

# Define the upper bound for outliers
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
filtered_df = allproperties[
    (allproperties['price'] <= upper_bound['price']) &
    (allproperties['bed'] <= upper_bound['bed']) &
    (allproperties['house_size'] <= upper_bound['house_size']) &
    (allproperties['acre_lot'] <= upper_bound['acre_lot']) &
    (allproperties['bath'] <= upper_bound['bath'])
]

In [48]:
# Minimum number of rows required (15,000 in this case)
min_rows = 13000

# Loop through unique states and create separate CSV files
unique_states = allproperties['state'].unique()
for state in unique_states:
    state_df = allproperties[allproperties['state'] == state]

    # Check the number of rows in the DataFrame
    num_rows = len(state_df)

    # Only save the file if it has over 15,000 rows
    if num_rows >= min_rows:
        output_file = os.path.join(output_dir, f'{state}_properties.csv')
        state_df.to_csv(output_file, index=False)
    else:
        print(f"Skipped {state}_properties.csv with {num_rows} rows (less than 15,000).")

Skipped Virgin Islands_properties.csv with 345 rows (less than 15,000).
Skipped Wyoming_properties.csv with 3 rows (less than 15,000).
Skipped West Virginia_properties.csv with 3 rows (less than 15,000).
Skipped Delaware_properties.csv with 1742 rows (less than 15,000).


In [51]:
allproperties.shape

(424426, 9)

In [52]:

#Function to generate unique IDs
def generate_ids(df):
    state = df['state'].iloc[0][:5].replace(' ', '')  # Get the first 4 letters of the state
    df['ID'] = state + df.groupby('state').cumcount().add(1).astype(str)
    return df.set_index('ID')  # Set the 'ID' column as the index

# Loop through the CSV files
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        # Read the CSV file into a DataFrame
        filepath = os.path.join(input_dir, filename)
        df = pd.read_csv(filepath)
        
        # Add the ID column and set it as the index
        df = generate_ids(df)
        
        # Save the DataFrame back to the CSV file with the ID as the index
        df.to_csv(filepath)

In [53]:
# Loop through the CSV files
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        # Read the CSV file into a DataFrame
        filepath = os.path.join(input_dir, filename)
        df = pd.read_csv(filepath)

        # Check the number of rows in the DataFrame
        num_rows = len(df)

        # Only fill NaN values for files with over 15,000 rows
        if num_rows >= min_rows:
            # Filter data with non-NaN 'house_size'
            valid_data = df.dropna(subset=['house_size'])

            # Separate features (bed, bath) and target (house_size)
            X = valid_data[['bed', 'bath']]
            y = valid_data['house_size']

            # Train a linear regression model
            model = LinearRegression()
            model.fit(X, y)

            # Predict 'house_size' for rows with NaNs
            nan_data = df[pd.isna(df['house_size'])]
            if not nan_data.empty:
                nan_X = nan_data[['bed', 'bath']]
                nan_data['house_size'] = model.predict(nan_X)

                # Round the predicted 'house_size' values to the nearest whole number
                nan_data['house_size'] = nan_data['house_size'].round()

                # Update the original DataFrame with filled 'house_size'
                df.update(nan_data)

                # Save the updated DataFrame back to the same CSV file
                df.to_csv(filepath, index=False)

In [54]:
# Loop through the CSV files
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        # Read the CSV file into a DataFrame
        filepath = os.path.join(input_dir, filename)
        df = pd.read_csv(filepath)

        # Check the number of rows in the DataFrame
        num_rows = len(df)

        # Only fill NaN values for files with over 15,000 rows
        if num_rows >= min_rows:
            # Filter data with non-NaN 'acre_lot'
            valid_data = df.dropna(subset=['acre_lot'])

            # Separate features ('house_size', 'bed', 'bath') and target ('acre_lot')
            X = valid_data[['house_size', 'bed', 'bath']]
            y = valid_data['acre_lot']

            # Train a linear regression model
            model = LinearRegression()
            model.fit(X, y)

            # Predict 'acre_lot' for rows with NaNs
            nan_data = df[pd.isna(df['acre_lot'])]
            if not nan_data.empty:
                nan_X = nan_data[['house_size', 'bed', 'bath']]
                nan_data['acre_lot'] = model.predict(nan_X)

                # Round the predicted 'acre_lot' values to one decimal place
                nan_data['acre_lot'] = nan_data['acre_lot'].round(1)

                # Update the original DataFrame with filled 'acre_lot'
                df.update(nan_data)

                # Save the updated DataFrame back to the same CSV file
                df.to_csv(filepath, index=False)

In [55]:
# Read the "uszips.csv" file into a DataFrame
uszips_df = pd.read_csv('Data/original_data/uszips.csv')

# Function to get latitude and longitude for a zip code
def get_lat_long(zip_code):
    try:
        # Find the corresponding row in uszips_df based on the zip code
        row = uszips_df[uszips_df['zip'] == int(zip_code)]
        if not row.empty:
            return row.iloc[0]['lat'], row.iloc[0]['lng']
    except Exception as e:
        pass
    return None, None

# Loop through the CSV files
for filename in os.listdir(input_dir):
    if filename.endswith('.csv'):
        # Read the CSV file into a DataFrame
        filepath = os.path.join(input_dir, filename)
        df = pd.read_csv(filepath)
        
        # Apply the get_lat_long function to each row to get latitude and longitude
        df['latitude'], df['longitude'] = zip(*df['zip_code'].map(get_lat_long))
        
        # Drop rows with NaN values in 'latitude' and 'longitude' columns
        df.dropna(subset=['latitude', 'longitude'], inplace=True)
        
        # Save the updated DataFrame back to the CSV file with latitude and longitude columns
        df.to_csv(filepath, index=False)

In [56]:
# Create a text file to store the NaN counts. This was used for understanding the dataset.
# Since the data set has been cleaned, it's no longer needed. Leaving it as a reference of our work.
output_file = 'nan_counts.txt'

with open(output_file, 'w') as f:
    # Loop through the CSV files
    for filename in os.listdir(input_dir):
        if filename.endswith('.csv'):
            # Read the CSV file into a DataFrame
            filepath = os.path.join(input_dir, filename)
            df = pd.read_csv(filepath)

            # Count NaNs for each column in the DataFrame
            nan_counts = df.isna().sum()

            # Get the total number of rows
            total_rows = len(df)

            # Write the NaN counts and total rows for each file to the text file
            f.write(f"NaN counts and total rows for {filename}:\n")
            for column in df.columns:
                f.write(f"{column}: {nan_counts[column]}\n")
            f.write(f"Total Rows: {total_rows}\n")
            f.write('\n')